<img src="images/housing_market.jpg_fit=scale" width=400 height=400 />

# King County, WA Housing Sales Analysis

# Overview
---

Our team was hired by a hot-shot real estate agency to create a model which can predict the price of a house in the King County area based on differnt features. Our analysis concluded that the best features for prediction of housing prices are the following: city/town/neighborhood location, square footage, number of bedrooms, whether it's on the water, how good of a view it has, the property condition, and the age of the property. 

To acheive our goal, we used multiple linear regression models to analyse housing sales in King County, WA using housing data gathered within the county from 2014 and 2015. For this analysis, we used Ordinary Least Squared, Train-Test Split, and K-Fold Cross Validation regression models to create an efficient predictive model. 

Our analysis shows that based on the R^2 scores, our regression model's accuracy is 80.1% with 95% conf interval +-3.9% 

# Business Understanding
---
Our stakeholder wants to be able to predict the price of a house based on certain features provided by their customers. 

Our project will answer the following questions:
* With what accuracy could we predict the price of a house based on these features?
* What specific housing features will provide us with the most accurate model? 

# Data Understanding
---
The data used for this project was sourced from a dataset called ‘King County House Sales’ and contains information regarding housing sales statistics in King County, WA.

##### The dataset contains the following columns:

* ```id```: A unique sale id relating to a house sale
* ```date```: Date of house sale
* ```price```: The price which the house sold for
* ```bedrooms```: How many bedrooms the house has
* ```bathrooms```: How many bathrooms the house has
* ```sqft_living```: How much square footage the house has
* ```sqft_lot```: How much square footage the lot has
* ```floors```: How many floors the house has
* ```waterfront```: Whether the house is on the waterfront. Originally contained ‘YES’ or ‘NO’, converted to 0 or 1 for comparative purposes
* ```view```: Whether the house has a view and whether it’s fair, average, good, or excellent. Converted to numberical (0-4) for comparative purposes
* ```condition```: overall condition of the house: Poor, Fair, Average, Good, Very Good
* ```grade```: Numerical grading for house
* ```sqft_above```: How much of the houses square footage is above ground
* ```sqft_basement```: How much of the square footage is in the basement
* ```yr_built```: Year the house was built
* ```yr_renovated```: Year the house was renovated, if applicable
* ```zipcode```: House zipcode
* ```lat```: House’s latitude coordinate
* ```long```: House’s longitude coordinate
* ```sqft_living15```: Average size of living space for the closest 15 houses
* ```sqft_lot15```: Average size of lot for the closest 15 houses

# Data Cleaning

### Importing required modules

In [ ]:
# Data manipulations
import pickle
import pandas as pd
import numpy as np
import math
import statsmodels.api as sm
from itertools import combinations
from statsmodels.formula.api import ols


#SKlearn
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_validate, cross_val_predict


#Visualisations
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

### Importing data

In [ ]:
df = pd.read_csv('data/kc_house_data.csv')

### Processing cleaning

In [ ]:
# Explore our data
display(df.head())
df.describe()

In [ ]:
# Lets convert condition codes to numerical values for the prediction model
# Create function for conditional labeling:
def condition_coding (condition):
    """
    This will take the condition from string format and transform it into a corresponding code in integer format
    Poor = 1, Fair = 2, Average = 3, Good = 4, Very Good = 5
    """
    if condition == 'Poor':
        condition_code = 1
    elif condition == 'Fair':
        condition_code = 2
    elif condition == 'Average':
        condition_code = 3
    elif condition == 'Good':
        condition_code = 4
    elif condition == 'Very Good':
        condition_code = 5
    return condition_code

# Apply function to dataframe
df["condition_code"] = df['condition'].map(condition_coding)


In [ ]:
# Lets convert grade codes to numerical values for the prediction model
# Create function for conditional labeling:
def grade_coding (grade):
    """
    This takes the grade in string format, splits it into a list of characters
    It then concatenates the first two characters from the list and strips the whitespace and turns the result into an integer
    We are left with a one or two digit integer correspondinng to the grade of the property
    """
    grade_list = list(grade)
    grade_code = int((grade_list[0] + grade_list[1]).strip())
    return grade_code

df['grade_code'] = df['grade'].map(grade_coding)

In [ ]:
# Resolve missing values of house renovation dates. The column mode is 0, we fill missing values with it.
df['yr_renovated'] = df['yr_renovated'].fillna(0)

In [ ]:
# Create house age feature.
# House age feature calculated the timeframe between transaction date and house construction date.
df['age'] = df['date'].map(lambda x: int(x[-4:])) - df['yr_built']
df.loc[df['age'] < 0, 'age'] = 0

In [ ]:
# Feature was house renovated or not. 
def renovated (year):
    """
    This returns a True / False value on whether a property has been renovated or not
    """
    if year == 0.0:
        return 0
    elif year > 0.0:
        return 1
    else:
        return 0
    
df['renovated'] = df['yr_renovated'].map(renovated)

In [ ]:
# This creates a column of float values stating how old the renovations are on a property
# If a property has not been renovated it will have a 0.0 value
df['age_of_renovations'] = df['date'].map(lambda x: int(x[-4:])) - df['yr_renovated']
df.loc[df['age_of_renovations'] == 2014, 'age_of_renovations'] = 1
df.loc[df['age_of_renovations'] == 2015, 'age_of_renovations'] = 1
df['age_of_renovations'] = df['age_of_renovations'].fillna(0)

In [ ]:
# The majority of values in waterfront column are - No. Fill the missing values with mode value if this column.
df['waterfront'] = df['waterfront'].fillna('NO')

# Convert waterfront column to numerical values for the prediction model 
df['waterfront_coded'] = df['waterfront'].map({'NO':0, 'YES':1})


In [ ]:
# The majority of values in view column are - None. Fill the missing values with mode value if this column.
df['view'] = df['view'].fillna('NONE')

# Label view column for the prediction model.
df['view_coded'] = df['view'].map({'NONE':0, 'FAIR':1, 'AVERAGE':2, 'GOOD':3, 'EXCELLENT':4})


In [ ]:
# Some values of sqft_basementa are missing. Find the missing values by subtracting sqft_above from sqft living.
df['sqft_basement'] = np.where(df['sqft_basement'] == '?', df['sqft_living'] - df['sqft_above'], df['sqft_basement'])
df['sqft_basement'] = df['sqft_basement'].astype('float')

In [ ]:
# Check the shape of finall dataframe without geo location data
df.shape

### Geo Locate to convert lat/long columns into Cities/Towns/Neighborhoods

In [ ]:
# location coordinates were scrapped from OSM in other jupyter notebook. 
# Load processed geo data from another notebook
with open('./data/Data_frame_geoloc.pickle', 'rb') as df_geo_data:
    df_geo = pickle.load(df_geo_data)


In [ ]:
# Lets check size of dataframe, drop all dublicated columns.
df_geo.shape
df_geo.drop(['id', 'lat', 'price', 'yr_built', 'sqft_living', 'sqft_lot'], axis = 1 , inplace = True)


In [ ]:
df_geo.shape

In [ ]:
# Merge 2 dataframes and check all columns.
df = pd.concat([df,df_geo], axis =1, verify_integrity = True )
df.info()

In [ ]:
# Investigate states and counties for noise.
print(df["state"].value_counts()) #all recods from Washington state
print(df["county"].value_counts()) # 4 records from different counties. 


In [ ]:
#Remove records from other counties
df.drop(df[df.county != "King County"].index, inplace = True)
df.shape
#Drop county and state columns to reduce the number of features
df.drop("state", axis = 1, inplace = True)
df.drop("county", axis = 1, inplace = True)


In [ ]:
df.shape

In [ ]:
# Investigate "city" column for missing values
print(f"The number of records with missing cities {sum(df.city.isna())}") #779 records have no cities in it. 
print(f"Missing values are in {round(sum(df.city.isna())/df.shape[0]*100,2)} % of data")

In [ ]:
#Remove records with missing cities
df.drop(df[df.city.isna() == True].index, axis = 0, inplace = True)
df.shape

In [ ]:
# Investigate "Type_place" column for missing values
print(f"The number of records with missing values {sum(df.Type_place.isna())}")

In [ ]:
# Investigate "suburb" column for missing values
print(f"The number of records with missing values {sum(df.suburb.isna())}")
print(f"Missing \"suburb\"values are in {round(sum(df.suburb.isna())/df.shape[0]*100,2)} % of data")
df_geo.suburb.value_counts() #There is no strong patterns in this data
print("Missing \"suburb\" values for cities are in ", round(sum(df[df.Type_place == "city"].suburb.value_counts())/df[df.Type_place == "city"].shape[0]*100,2), "% of records")
# We won't proceed with this data


In [ ]:
# Remove outliers in bedrooms
df.drop(df[(df.bedrooms == 33)].index, axis = 0, inplace = True)
df.drop(df[(df.bedrooms == 11)].index, axis = 0, inplace = True)
df.drop(df[(df.bedrooms == 10)].index, axis = 0, inplace = True)

In [ ]:
# Drop all cathegorical values that were coverted to numerical
to_drop_cat = ["date", "waterfront", "view", "condition", "grade"]
df.drop(to_drop_cat, axis = 1, inplace = True)

In [ ]:
#reset indexes
df_fin = df.reset_index().drop("index", axis = 1)

In [ ]:
# Different type of geo columns
Geo_columns_basic_all = ["Type_place", "city"]
Geo_columns_basic_type = ["Type_place"]
Geo_columns_basic_city_names = ["city"]
Geo_columns_advanced = ["suburb"]
Geo_columns_drop = ["To_drop_place_ID", "To_drop_road"]

In [ ]:
# Setup the type of geo columns that will be used for modelling
# Model one : encode each city into our dataframe
modeling_columns = Geo_columns_basic_city_names
Geo_columns_drop = [column for column in list(df_fin.columns) if ((column not in modeling_columns)  and (column != "id"))]

#seting up data categorical data for encoding !!!!!! (need to change later on)
X_cat_geo = df_fin.drop(Geo_columns_drop, axis = 1)


In [ ]:
# setup One Hot Encoder 
encoder_geo_basic = OneHotEncoder(handle_unknown = "ignore")
fit_df = X_cat_geo.drop("id", axis = 1)
encoder_geo_basic.fit(fit_df)

In [ ]:
# Prepare transformed dataset
X_cat_transf=encoder_geo_basic.transform(fit_df)
X_geo_df = pd.DataFrame(X_cat_transf.todense(), columns = encoder_geo_basic.get_feature_names())
#X_geo_df["id"] = X_cat_geo["id"]
df_cities_columns = encoder_geo_basic.get_feature_names() # New added column names. If you need them later on.
df_cities = pd.concat([df_fin, X_geo_df], axis = 1)       # DF with coded cities/towns/villages


In [ ]:
# Setup the type of geo columns that will be used for modelling
# Model two : encode only size of city into model
modeling_columns = Geo_columns_basic_type
Geo_columns_drop = [column for column in list(df_fin.columns) if ((column not in modeling_columns)  and (column != "id"))]

#seting up data categorical data for encoding !!!!!! (need to change later on)
X_cat_geo = df_fin.drop(Geo_columns_drop, axis = 1)


In [ ]:
# setup One Hot Encoder 
encoder_geo_basic = OneHotEncoder(handle_unknown = "ignore")
fit_df = X_cat_geo.drop("id", axis = 1)
encoder_geo_basic.fit(fit_df)

In [ ]:
# Prepare transformed dataset
X_cat_transf=encoder_geo_basic.transform(fit_df)
X_geo_df = pd.DataFrame(X_cat_transf.todense(), columns = encoder_geo_basic.get_feature_names())
#X_geo_df["id"] = X_cat_geo["id"]
df_types_columns = encoder_geo_basic.get_feature_names() # New added column names. If you need them later on.
df_types = pd.concat([df_fin, X_geo_df], axis = 1)       # DF with coded cities/towns/villages
df_types

In [ ]:
df_cities.info()

In [ ]:
# Drob geo cathegorical columns that were transfered to numerical.
to_drop_index = [(number) for number, column in enumerate(df_cities.dtypes) if column == 'O']
to_drop_columns = list(df_cities.columns[to_drop_index])
df_cities = df_cities.drop(to_drop_columns, axis =1)

to_drop_index = [(number) for number, column in enumerate(df_types.dtypes) if column == 'O']
to_drop_columns = list(df_types.columns[to_drop_index])
df_types = df_types.drop(to_drop_columns, axis =1)



In [ ]:
#Final clean_up
final_drop = ["zipcode", "id", "zipcode", "lat", "long", "lon","To_drop_place_ID"]
df_cities = df_cities.drop(final_drop, axis = 1)
df_types = df_types.drop(final_drop, axis = 1)

In [ ]:
# Output from data cleaning: 2 dataframes, cleaned from noise and with corrected data.
# df_cities - dataframe with encoded cities where property is located
# df_types - dataframe with encoded type of location (city, town or village)

df_cities
df_types
df_cities.info()

In [ ]:
# df_cities is our final merged df which we will base our modeling off of
df_cities.head()

# Data Modeling

## Analysing Regression with Property Feature Columns

We want to look data corresponding to the property features: square footage, number of bedrooms, number of bathrooms, view quality, and waterfront location

In [ ]:
# create a correlation heat map for all the columns relating to house features (sqft, bedrooms, floors, etc)
# checks if there are any features that might cause collinearity issues

cols = ['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront_coded', 'view_coded', 'sqft_above', 'sqft_basement', 'sqft_living15', 'sqft_lot15']
corr = df_cities[cols].corr()

fig, ax = plt.subplots(figsize=(12,10)) 
sns.heatmap(corr, annot=True, ax=ax);


In [ ]:
# normalize and numerical columns that dont contain ordinal or categorical numbers
col_norm = ['sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement', 'sqft_living15', 'sqft_lot15']

def norm(series):
    return (series - series.mean())/series.std()
for feat in col_norm:
    df_cities[feat+"_norm"] = norm(df_cities[feat])

In [ ]:
# this function creates an ordinary least squares model for price using input columns and data frame 
def ols_model(cols, df):
    predictors = '+'.join(cols)
    formula = 'price' + '~' + predictors
    model = ols(formula=formula, data=df).fit()
    return model

In [ ]:
# using an ols model to look at p-values and R^2 values for all of the feature columns
col_pred = ['bedrooms', 'bathrooms', 'sqft_living_norm', 'sqft_lot_norm', 'floors', 'waterfront_coded', 'view_coded', 'sqft_above_norm', 'sqft_basement_norm', 'sqft_living15_norm', 'sqft_lot15_norm']

ols_model(col_pred, df_cities).summary()


### Multicollinearity Issues
---

There are potential multicollinearity issues with **sqft_above** and **sqft_living**, **sqft_living** and **bathrooms**, **sqft_living15** and **sqft_living**, and **sqft_lot15** and **sqft_lot** - this is due to high correlation values

**bathrooms** and **sqft_basement** have proven to not be statistically significant (p > 0.05)

Best option to remove **sqft_above**, **sqft_basement**, **sqft_living15**, **sqft_lot15**, and **bathrooms** columns to avoid any potential modeling issues

**sqft_lot** and **floors** did not have a significant effect on the R^2 value for the ols model, so we have removed these column from the model as well

In [ ]:
# finalized list of features to be used for regression
cols_features = ['bedrooms', 'sqft_living_norm', 'waterfront_coded', 'view_coded']


In [ ]:
# below is the ols model for the final selection of columns for house features
# chose these columns based on p-value and how each effected R^2 value

ols_model(cols_features, df_cities).summary()

In [ ]:
# drop the unused feature columns from the data frame to continue with out regression modeling
drop_cols = ['sqft_lot', 'sqft_above', 'sqft_basement', 'bathrooms', 'floors', 'sqft_living', 'sqft_living15', 'sqft_lot15', 'sqft_lot15_norm', \
            'sqft_living15_norm', 'sqft_lot_norm', 'sqft_above_norm', 'sqft_basement_norm']
df_cities = df_cities.drop(drop_cols, axis=1)


In [ ]:
df_cities.columns

In [ ]:
# create a train test split model to output the R^2 value and the mean squared error for the house feature columns
X = df_cities[cols_features]
y = df_cities['price']

#create train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

#create model
poly = PolynomialFeatures(2)
X_poly = poly.fit_transform(X_train)
model = LinearRegression()
model.fit(X_poly, y_train)

#predict model
predict_train = model.predict(X_poly)
predict_test = model.predict(poly.transform(X_test))
mse = mean_squared_error(y_test, predict_test)

#score model
train_score = model.score(X_poly, y_train)
test_score = model.score(poly.transform(X_test), y_test)

train_score, test_score, mse


print(f'Train Score: {train_score}\nTest Score: {test_score}\nMean Squared Error: {mse}')

## Analysing Regression With Property Condition Data

Looking at the data corresponding to the condition of the property we subset the following columns:
'condition_code', 'grade_code', 'age', 'age_of_renovations', and 'renovated'

We then look at correlations between these variables:

In [ ]:
# First I'll crate some basic variables that will be use through the analysis and modeling
df_condition = df_cities[['price','condition_code','grade_code','age','age_of_renovations','renovated']]
X_condition = df_condition[['condition_code','grade_code','age','age_of_renovations','renovated']]
y_condition = df_condition.price

In [ ]:
# Now lest look at the feature correlations
X_condition.corr()

In [ ]:
# We can view this as a heat map to give a better view
plt.figure(figsize=(8,6))
sns.heatmap(X_condition.corr(), annot=True)
plt.show()

We can see that 'age_of_renovations' is closely correlated with 'renovated' and may cause multicolinearity issues. Indeed they are related to the same characteristsic and tell the same tale. We can eliminate one of these from our analysis.

If a property is showing a value > 1 in 'age_of_renovations' then we know it has been renovated which tells us the same as the 'renovated' column... so we will drop 'renovated'

In [ ]:
formula_cond = 'price ~ condition_code + grade_code + age + age_of_renovations'

condition_model = ols(formula=formula_cond, data=df_condition).fit()
condition_model_summ = condition_model.summary()

print(condition_model_summ)

Lets take a look at some regressions and see what will give us the strongest model based on the condition variables

Using all of our features, we get a strong score on both a training data set and also the test set

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_condition, 
                                                    y_condition,
                                                    test_size=None,
                                                    random_state=42
                                                   )

lr = LinearRegression()
lr.fit(X_train, y_train)
lr.score(X_train, y_train)
lr.score(X_test, y_test)

print(lr.score(X_train, y_train))
print(lr.score(X_test, y_test))

We don't need all of these features though so lets iterate through some linear regressions with different feature combinations and see what will give us the simplest, but highest scoring model

In [ ]:
# dropping the 'renovated' feature that was giving us multicolinearity issues

X_condition_1 = df_condition[['condition_code','grade_code','age','age_of_renovations']]
X_train, X_test, y_train, y_test = train_test_split(X_condition_1, 
                                                    y_condition,
                                                    test_size=None,
                                                    random_state=42
                                                   )

lr = LinearRegression()
lr.fit(X_train, y_train)

print(lr.score(X_train, y_train))
print(lr.score(X_test, y_test))

Conditon and Grade tell a similar story about a property. Looking at he data we see Grade is a lot more in depth and provides more bins, so it is potentially more powerful. Lets drop Condition.

In [ ]:
# dropping the 'condition_code' feature from the model

X_condition_2 = df_condition[['grade_code','age','age_of_renovations']]
X_train, X_test, y_train, y_test = train_test_split(X_condition_2, 
                                                    y_condition,
                                                    test_size=None,
                                                    random_state=42
                                                   )

lr = LinearRegression()
lr.fit(X_train, y_train)

print(lr.score(X_train, y_train))
print(lr.score(X_test, y_test))

That barely chaged our model score, and indeed when we look at just Condition it doesn't score well.

In [ ]:
X_condition_3 = df_condition[['condition_code']]
X_train, X_test, y_train, y_test = train_test_split(X_condition_3, 
                                                    y_condition,
                                                    test_size=None,
                                                    random_state=42
                                                   )

lr = LinearRegression()
lr.fit(X_train, y_train)

print(lr.score(X_train, y_train))
print(lr.score(X_test, y_test))

However, looking at just Grade we can see that it scores well and is indeed our strongest feature.

In [ ]:
X_condition_4 = df_condition[['grade_code']]
X_train, X_test, y_train, y_test = train_test_split(X_condition_4, 
                                                    y_condition,
                                                    test_size=None,
                                                    random_state=42
                                                   )

lr = LinearRegression()
lr.fit(X_train, y_train)

print(lr.score(X_train, y_train))
print(lr.score(X_test, y_test))

Including the 'age_of_renovations' feature does not add much to the model, so we will drop that as well

In [ ]:
X_condition_5 = df_condition[['grade_code','age_of_renovations']]
X_train, X_test, y_train, y_test = train_test_split(X_condition_5, 
                                                    y_condition,
                                                    test_size=None,
                                                    random_state=42
                                                   )

lr = LinearRegression()
lr.fit(X_train, y_train)

print(lr.score(X_train, y_train))
print(lr.score(X_test, y_test))

*Interestingly, Age does not score well by itself. However, when coupled with the Grade feature it gives a large improvement to the overall score.

This combination of features gives us the highest score, with the least features.

In [ ]:
X_condition_6 = df_condition[['age']]
X_train, X_test, y_train, y_test = train_test_split(X_condition_6, 
                                                    y_condition,
                                                    test_size=None,
                                                    random_state=42
                                                   )

lr = LinearRegression()
lr.fit(X_train, y_train)

print(lr.score(X_train, y_train))
print(lr.score(X_test, y_test))

In [ ]:
X_condition_7 = df_condition[['grade_code','age']]
X_train, X_test, y_train, y_test = train_test_split(X_condition_7, 
                                                    y_condition,
                                                    test_size=None,
                                                    random_state=42
                                                   )

lr = LinearRegression()
lr.fit(X_train, y_train)

print(lr.score(X_train, y_train))
print(lr.score(X_test, y_test))

Indeed, these are the two columns we will include on the overall "Linear Regression" model

Lets now look at some Polynomial regressions on the "condition" data and see if we can improve on our model

In [ ]:
X_conditions = [X_condition, X_condition_1, X_condition_2, X_condition_3, X_condition_4, X_condition_5, X_condition_6, X_condition_7]

In [ ]:
# A function thet interates through our combinations to determine the the best number of Polynomial Features
# It returns an array of train and test scores for each combination

def poly_scores_array (X_lst, y):
    poly_scores = []
    for X in X_lst:
        train_scores = []
        test_scores = []
        for i in range(1,7):
            poly = PolynomialFeatures(i)
            X_poly = pd.DataFrame(poly.fit_transform(X))
            X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=None, random_state=42)
            model = LinearRegression()
            model.fit(X_train, y_train)
            train_scores.append(model.score(X_train, y_train))
            test_scores.append(model.score(X_test, y_test))
            scores = (train_scores, test_scores)
            poly_scores.append(scores)
    return poly_scores

#uncomment code to see output array
#poly_scores_array(X_conditions, y_condition)

Just eyeballing the array we conistently get our highest scores up to 4 polynomial features and then our test scores drop significantly.

In [ ]:
# Having determined the best number of polynomial features is 4 for the property condition data
# Create a function to iterate through our feature combinations to determine the best model

def best_poly_model (X_lst, y):
    poly_model_scores = []
    for X in X_lst:
        poly_2 = PolynomialFeatures(4)
        X_poly = pd.DataFrame(poly_2.fit_transform(X))
        X_train, X_test, y_train, y_test = train_test_split(X_poly, y,
                                                    test_size=None,
                                                    random_state=42)
        lr_poly = LinearRegression()
        lr_poly.fit(X_train, y_train)
        train_score = lr_poly.score(X_train, y_train)
        test_score = lr_poly.score(X_test, y_test)
        score = (train_score, test_score)
        poly_model_scores.append(score)
    return poly_model_scores

best_poly_model (X_conditions, y_condition)

The array above corresponds to our feature combinations. In our linear regression analysis we determined that X_condition_7 gave us our best model and we can see above that the same combination gives us excellent results and even scores the best on the test data overall.

Lets see this isolated below

In [ ]:
poly_7 = PolynomialFeatures(4)
X_poly_7 = pd.DataFrame(poly_7.fit_transform(X_condition_7))
X_train, X_test, y_train, y_test = train_test_split(X_poly_7, y_condition,
                                                    test_size=None,
                                                    random_state=42)
lr_poly_7 = LinearRegression()
lr_poly_7.fit(X_train, y_train)

print(lr_poly_7.score(X_train, y_train))
print(lr_poly_7.score(X_test, y_test))

After performing both linear and ploynomial regression analysis on the Property Condition data, we have determined that _Grade (as coded in 'gradecode') and Age are the best variables to use in our model

In [ ]:
cols_to_drop = ['yr_built', 'yr_renovated', 'condition_code', 'renovated', 'age_of_renovations']
df_cities = df_cities.drop(cols_to_drop, axis=1)

In [ ]:
df_cities.columns

### Regression Model for Property Feature Columns and Property Condition Columns

Now that we have narrowed down our property feature columns and property condition columns we can create a new regression model

In [ ]:
cols_feat_cond = ['bedrooms', 'sqft_living_norm', 'waterfront_coded', 'view_coded', 'grade_code', 'age']

In [ ]:
# create a train test split model to output the R^2 value and the mean squared error 
# for the property feature columns combined with property condition columns
X = df_cities[cols_feat_cond]
y = df_cities['price']

#create train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

#create model - polynomial 2 is the best fit for this data (highest scores and lowest mean squared error)
poly = PolynomialFeatures(2)
X_poly = poly.fit_transform(X_train)
model = LinearRegression()
model.fit(X_poly, y_train)

#predict model
predict_train = model.predict(X_poly)
predict_test = model.predict(poly.transform(X_test))
mse = mean_squared_error(y_test, predict_test)

#score model
train_score = model.score(X_poly, y_train)
test_score = model.score(poly.transform(X_test), y_test)

train_score, test_score, mse


print(f'Train Score: {train_score}\nTest Score: {test_score}\nMean Squared Error: {mse}')

As we can see the R^2 scores for our new model is better than both the Property Features Model and the Property Condition Model. This shows us that are new data better fits our regression line!

Now it's time to add in our location data for our final regression model!

# Regression Results

Below we have created a linear regression calculator which will output our R^2 value (model accuracy) as well as the mean squared error for our finalized dataset. It will also print out a random sample of our predictive model

In [ ]:
# function for modeling linear regression 
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

def linear_reg_calculator(Data, n = 1, to_drop = []):
    #model preparation
    Data = Data.reset_index().drop("index", axis = 1)
    poly = PolynomialFeatures(n)
    linreg = LinearRegression()
    scoring_model = ["r2", "neg_mean_squared_error"]
    kf = KFold(n_splits=5)
    y = Data["price"]
    X = Data.drop("price", axis = 1)    
    # execute code that will generate warnings
    # Fit and transform X for polynomial function
    if n > 1:
        X_new = X.drop(df_cities_columns, axis = 1)    # Drop geo categorical data
        if to_drop != []:
            X_new2 = X_new.drop(to_drop, axis = 1)     # Drop any additional columns we want
            X_new = X_new2  
        else:
            pass
        X_poly= poly.fit_transform(X_new)                                        # Poly transformation on numerical data
        X = pd.concat([pd.DataFrame(X_poly), Data[df_cities_columns]], axis = 1) # Making new dataframe
        
    
    else:
        pass
    
    # Modelling
    reg_poly = cross_validate(linreg, X, y, scoring= scoring_model, cv = kf)
    mean = np.mean(reg_poly["test_r2"])
    stand = np.std(reg_poly["test_r2"])
    mean_mean_sq_error = -np.mean(reg_poly["test_neg_mean_squared_error"])
    
    # Random house experiment
    difference_list=[]
    random_sample = np.random.randint(0, len(y))                         # Choose the random house from our prediction
    y_hat = cross_val_predict(linreg, X, y, cv = kf)                     # Make predictions based on our model 
    price = round(y[random_sample],0)                                    # sample house price
    predicted_price = round(y_hat[random_sample], 0)                     # sample house predicted price
    difference = round(100*(y_hat[random_sample]/y[random_sample]-1),2)  # difference between prices
    
    # Random 20 houses prediction
    for i in list(range(20)):
        random_sample = np.random.randint(0, len(y))
        difference_list.append(round(100*(y_hat[random_sample]/y[random_sample]-1),2))
    difference_mean = round(np.mean(difference_list),2)    
    difference_std = round(np.std(difference_list),2)    
    
    #Outputs
    print(f"Model accuracy based on R2 score {round(mean*100,1)}% with 95% conf interval +-{round(2*stand*100,1)}%")
    print(f"Mean Squared Error {round(mean_mean_sq_error, 0)}")
    print(f"Random house price {price}, predicted price {predicted_price}, difference {difference} %")
    print(f"Sample(20 houses) prediction price difference {difference_mean} %")
    return mean_mean_sq_error, mean, y_hat, y
    
#Work example
#(MSE, mean_ac, y_hat_regr, y_reg) = linear_reg_calculator(df_cities, 2, [])

In [ ]:
(MSE, mean_ac, y_hat_regr, y_reg) = linear_reg_calculator(df_cities, 2)

In [ ]:
# Linear regression performance plot.
def currency(x, pos):
    """The two args are the value and tick position"""
    if x >= 1e9:
        s = '${:1.1f}B'.format(x*1e-9)
    elif x >= 1e6:
        s = '${:1.1f}M'.format(x*1e-6)
    else:
        s = '${:1.0f}K'.format(x*1e-3)
    return s

x_line = np.linspace(0,2000000)
fig,axs = plt.subplots(figsize = (12,8))
sns.scatterplot(y_reg,y_hat_regr , marker = "." , s = 8,alpha = 1, label = "Individual house sell price")
axs.plot(x_line, x_line, color ="red", label = "Ideal prediction line")
axs.set_xlim(0,2000000) ; axs.set_ylim(0,2000000)
axs.yaxis.set_major_formatter(currency)
axs.xaxis.set_major_formatter(currency)
axs.set_aspect("equal")
axs.set_title("House Price vs house predicted price plot")
axs.set_xlabel("House sell price")
axs.set_ylabel("House predicted sell price")
axs.legend();
plt.savefig("./images/PricevsPredict_scatter.png")

# Conclusion
---
Our final linear regression model can conclude an accuracy of %80.1 based on the R^2 score. 

Our House Predicted Price scatter plot shows how well our predicted regression model fits the individual selling prices of houses

This data tells us that with our selected feature columns (neighborhood, age, grade code, bedrooms, living square footage, waterfront, and view quality) we have created a model that can predict house prices based on features with an %80.1 accuracy